In [ ]:
import os
import numpy as np
import scipy.sparse as sp
import scipy.sparse.csgraph as csgraph
import nibabel as nib
import pyvista as pv
from nilearn import plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [ ]:
SURF_DIR = '../../datasets/NMT_v2.0_sym/NMT_v2.0_sym_surfaces'

left_inflated = os.path.join(SURF_DIR, 'lh.mid_surface.inf_300.rsl.gii')
right_inflated = os.path.join(SURF_DIR, 'rh.mid_surface.inf_300.rsl.gii')

x = nib.load(left_inflated) 
coords = x.agg_data('NIFTI_INTENT_POINTSET')
faces = x.agg_data('NIFTI_INTENT_TRIANGLE')

faces_pv = np.hstack([np.full((faces.shape[0], 1), 3), faces]).astype(np.int64).ravel()
mesh = pv.PolyData(coords, faces_pv)

In [ ]:
def geodesic_ball(mesh, seed, radius_mm=4.0):
    pts = mesh.points
    f = mesh.faces.reshape(-1, 4)[:, 1:]  # (n_faces, 3)

    # build undirected edges from triangles
    e01 = f[:, [0, 1]]
    e12 = f[:, [1, 2]]
    e20 = f[:, [2, 0]]
    edges = np.vstack([e01, e12, e20])
    edges = np.sort(edges, axis=1)
    edges = np.unique(edges, axis=0)

    # edge weights = euclidean length along the surface edges
    w = np.linalg.norm(pts[edges[:, 0]] - pts[edges[:, 1]], axis=1)

    # sparse adjacency
    n = pts.shape[0]
    A = sp.csr_matrix((w, (edges[:, 0], edges[:, 1])), shape=(n, n))
    A = A + A.T

    dist = csgraph.dijkstra(A, directed=False, indices=seed)
    return (dist <= radius_mm).astype(np.uint8)

In [ ]:
# --- user knobs ---
ap_axis = 1                 # usually y in template space; verify once by rotating
ap_targets = {
    # body patches
    'MiddleBody':     6.3,
    'AnteriorBody':  11.3,

    # face patches
    'MiddleFace':     8.0,   # center of ~7–9 mm range
    'AnteriorFace':  13.6,

    # object patches
    'MiddleObject':   9.0,
    'AnteriorObject': 12.8,

    # color patches
    'MiddleColor':    8.3,
    'AnteriorColor': 14.5,

    # scene-selective (parahippocampal / retrosplenial-ish)
    'Scene':          5.3,

    # control / undefined
    'Unknown':        7.3,
}
ap_tol_mm = 0.75            # slice thickness-ish; widen if you get too few verts
restrict_axis = 0           # left/right axis; usually x
restrict_mode = 'abs'       # 'abs' keeps near midline; 'sign' keeps hemisphere side
restrict_val = None         # set e.g. -1 for left hemi, +1 for right hemi if using 'sign'
restrict_tol = 25.0         # mm window if using restrict_mode='abs' (keeps ventral-ish/midline-ish)

In [ ]:
points = {}
for name, ap_mm in ap_targets.items():
    # vertices near target ap
    ap = coords[:, ap_axis]
    cand = np.where(np.abs(ap - ap_mm) <= ap_tol_mm)[0]
    if cand.size == 0:
        raise ValueError(f'no vertices near ap={ap_mm} (tol={ap_tol_mm} mm)')

    # left hemi: avoid medial wall by preferring more lateral vertices (more negative x)
    xcoord = coords[cand, 0]
    zcoord = coords[cand, 2]

    # keep lateral-ish subset (20% most lateral for left hemi)
    x_thr = np.quantile(xcoord, 0.2)
    cand2 = cand[xcoord <= x_thr]

    # within that, pick a ventral-ish vertex (small z)
    j = np.argmin(coords[cand2, 2])
    vidx = int(cand2[j])
    points[name] = {'vidx': vidx, 'pt': coords[vidx]}
points

In [ ]:
# roi -> family
roi_family = {
    'MiddleBody':     'body',
    'AnteriorBody':   'body',

    'MiddleFace':     'face',
    'AnteriorFace':   'face',

    'MiddleObject':   'object',
    'AnteriorObject': 'object',

    'MiddleColor':    'color',
    'AnteriorColor':  'color',

    'Scene':          'scene',
    'Unknown':        'unknown',
}

# family -> integer label (0 = background)
family_label = {
    'background': 0,
    'face':       1,
    'body':       2,
    'object':     3,
    'color':      4,
    'scene':      5,
    'unknown':    6,
}

# label -> color (ordered by label index)
cmap = [
    'lightgrey',  # 0 background
    'dodgerblue', # 1 face
    'limegreen',  # 2 body
    'orange',     # 3 object
    'gold',       # 4 color
    'purple',     # 5 scene
    'dimgray',    # 6 unknown
]


In [ ]:
roi_radius = 1
labels = np.zeros(mesh.n_points, dtype=np.uint8)

for roi, ap_mm in ap_targets.items():
    fam = roi_family[roi]
    lab = family_label[fam]

    vidx = points[roi]['vidx']  # you already computed these
    mask = geodesic_ball(mesh, vidx, radius_mm=roi_radius).astype(bool)

    # overwrite is intentional: later ROIs of same family just reinforce
    labels[mask] = lab

mesh.point_data['roi_family'] = labels

In [ ]:
pv.set_jupyter_backend('trame')

pl = pv.Plotter()
pl.add_mesh(
    mesh,
    scalars='roi_family',
    cmap=cmap,
    clim=[0, len(cmap) - 1],
    categories=True,
    smooth_shading=False,
    interpolate_before_map=False,
)
pl.reset_camera()
pl.show()

In [ ]:
## PLOT BOTH HEMIS
fig = plt.figure(figsize=(10, 4))

ax1 = fig.add_subplot(1, 2, 1, projection='3d')
plotting.plot_surf(
    surf_mesh=str(left_inflated),
    hemi='left',
    view='lateral',
    axes=ax1,
)
ax1.set_title('left inflated')

ax2 = fig.add_subplot(1, 2, 2, projection='3d')
plotting.plot_surf(
    surf_mesh=str(right_inflated),
    hemi='right',
    view='lateral',
    axes=ax2,
)
ax2.set_title('right inflated')

plt.tight_layout()
plt.show()